## Agentic Travel Planner Chatbot

- This application utilizes the **Gemini API** to function as a sophisticated travel planner.
- Takes detailed traveler information and generating a comprehensive, strictly-formatted trip itinerary.
- The user interface is built using Gradio, providing a convenient chat environment.
- The final itinerary which the user is happy with, can be saved directly to a file via the model's tool-calling capability.

### Key Features

1. **Strict Output Generation:** Uses a detailed system prompt to force the LLM to provide 17 specific pieces of information for every itinerary.
2. **Contextual Planning:** Reads traveler details from a travel_summary.txt file to ensure the itinerary is tailored to specific interests.
3. **Gradio Chat UI:** Provides a simple, interactive chat interface for itinerary refinement.
4. **Tool-Calling for Persistence:** Implements a function tool that the LLM can call to save the final generated itinerary to a file once the user is satisfied.

### Prerequisites:

1. You need a Gemini API key. This key should be set as an environment variable named GEMINI_API_KEY. 
2. Create summary.txt file. This file holds the context the model uses for planning. It is read once at startup.

**Example travel_summary.txt as below:**

- Vacation type: Family
- Kids: One 4 year boy
- Meals: Vegeterian
- Interests: Walking, Hiking, Kids friendly walking trails, Kids friendly parks and activities, city exploration, beach, reading, pubs, cafes, historical places, Artistic and handmade items

### Sample User prompts
- First prompt: We are going to Barcelona in December during Christmans for a week. Can you plan my trip?
- Second prompt: I am happy with your response. Save this to a file called trip.txt.

In [4]:
from dotenv import load_dotenv
from openai import OpenAI
import os
import json
import gradio as gr


In [5]:
load_dotenv(override=True)

True

In [6]:
google_api_key = os.getenv('GOOGLE_API_KEY')

In [ ]:
summary = ""
with open("me/travel_summary.txt", "r") as f:
    summary = f.read()

In [8]:
system_prompt = f"""You operate as a Travel Planning Agent. 
You are given specific traveller profiles and interests in the input variable {summary}.

MANDATORY REQUIREMENTS:

Utilization of Information: You MUST incorporate the information regarding the travellers and their interests, as provided in {summary}, into the planning of the itinerary.

Output Structure: Your response MUST contain a dedicated section for EACH of the following topics. 
If information for a section (e.g., address, news) is not provided, you must state that the information is "Not Provided" or "Not Applicable" (e.g., if no address is given, state "Distance from Address: Not Provided").

MANDATORY CONTENT SECTIONS (MUST BE INCLUDED):

Airport Transfer Plan: Detail the journey from the airport to the accommodation. MUST include suggested booking sites for tickets.
Weather Forecast: Provide the expected weather conditions for the travel period.
Essential Packing List: List critical items the travellers must carry.
Places to Visit: List specific attractions. MUST include the distance from the accommodation address (if provided) and the best mode of transport from that address.
Advance Booking Attractions: List all attractions that require or are highly recommended for advance ticket booking.
Budget Travel Passes: Identify and detail any cheap travel passes or day passes available.
Souvenir Shopping: Specify where to purchase authentic artistic souvenirs.
Local Dining: Recommend the best restaurants in the area.
Train Schedule (Airport): Provide train timings and frequency for travel to and from the airport.
Train Ticket Information: Detail where and how to purchase train tickets.
Local Transit Discounts: Detail available local travel passes and discounts (excluding the airport train).
Cultural Reading Suggestions: Recommend fiction and non-fiction book titles related to the local culture.
Media Suggestions: Recommend movies and/or music relevant to the visited location.
Local Phrases: List common phrases or local slang for greetings and basic interactions.
Local Alcoholic Beverage: Suggest a characteristic local alcoholic drink.
Local News/Events: Report any recent or relevant local news or major events in the area.
Local Activities: Suggest activities recommended by residents of the area. """

In [9]:
def save_to_file(content, filename):
    with open(filename, "w") as f:
        f.write(content)
    return {"recorded": "ok"}

In [10]:
save_to_file_json = {
    "name": "save_to_file",
    "description": "Call this ONLY after the user explicitly confirms they are happy with the content and want to save it. Requires the full content and the desired filename.",
    "parameters": {
        "type": "object",
        "properties": {
            "content": {"type": "string", "description": "The complete, final text (the LLM's response) that the user is satisfied with and wants to save."},
            "filename": {"type": "string", "description": "The desired name of the file"}
        }
    }
}

In [11]:
tools = [{"type": "function", "function": save_to_file_json}]

In [12]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
        
    return results

In [13]:
def chat(message, history):
    google = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
    model_name = "gemini-2.0-flash"
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    done = False
    while not done:
        response = google.chat.completions.create(model=model_name, messages=messages, tools=tools)
        finish_reason = response.choices[0].finish_reason
        print(finish_reason)
        if finish_reason == "tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            print(tool_calls)
            result = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(result)
        else:
            done = True

    return response.choices[0].message.content

In [15]:

gr.ChatInterface(
    fn=chat, 
    title="Travel Planner",
    type="messages",
    description="Ask anything about the trip").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
